In [ ]:
# Import required libraries
import akshare as ak
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Display settings for better output
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("✅ Libraries imported successfully!")
print(f"📊 AKShare version: {ak.__version__}")
print(f"🐼 Pandas version: {pd.__version__}")
print(f"🔢 NumPy version: {np.__version__}")


In [ ]:
# Get real-time data for all A-share stocks
print("📊 Fetching real-time stock data...")
real_time_data = ak.stock_zh_a_spot_em()

print(f"✅ Successfully retrieved {len(real_time_data)} stocks")
print(f"📈 Data shape: {real_time_data.shape}")
print(f"📅 Data columns: {list(real_time_data.columns)}")

# Display first 5 rows
print("\n🔍 First 5 stocks:")
real_time_data.head()


In [ ]:
# Example: Find specific stocks (like 平安银行 - Ping An Bank)
print("🔍 Searching for specific stocks...")

# Search by stock code
ping_an_bank = real_time_data[real_time_data['代码'] == '000001']
print("\n🏦 平安银行 (Ping An Bank - 000001):")
if not ping_an_bank.empty:
    stock = ping_an_bank.iloc[0]
    print(f"   📛 Name: {stock['名称']}")
    print(f"   💰 Current Price: {stock['最新价']} CNY")
    print(f"   📊 Change: {stock['涨跌幅']}%")
    print(f"   🔄 Volume: {stock['成交量']:,}")
    print(f"   💵 Amount: {stock['成交额']:,.0f} CNY")
else:
    print("   ❌ Stock not found")

# Search by name (contains keyword)
print("\n🔍 Searching stocks with '银行' (Bank) in name:")
bank_stocks = real_time_data[real_time_data['名称'].str.contains('银行', na=False)]
print(f"   📈 Found {len(bank_stocks)} bank stocks")
bank_stocks[['代码', '名称', '最新价', '涨跌幅', '成交量']].head(10)


In [ ]:
# Example: Get top performers (highest gains)
print("🚀 Top 10 gainers today:")
top_gainers = real_time_data.nlargest(10, '涨跌幅')
display_cols = ['代码', '名称', '最新价', '涨跌幅', '成交量']
print(top_gainers[display_cols].to_string(index=False))


In [ ]:
# Get detailed information for a specific stock
symbol = '000001'  # 平安银行
print(f"📋 Getting detailed information for stock {symbol}...")

try:
    stock_info = ak.stock_individual_info_em(symbol=symbol)
    print(f"✅ Successfully retrieved information for {symbol}")
    
    # Display the raw data
    print("\n📊 Raw stock information:")
    print(stock_info)
    
    # Convert to a more readable format
    print("\n📋 Formatted stock information:")
    info_dict = {}
    for _, row in stock_info.iterrows():
        info_dict[row['item']] = row['value']
    
    # Display key information
    key_info = {
        '股票代码': symbol,
        '股票名称': info_dict.get('股票名称', 'N/A'),
        '所属行业': info_dict.get('所属行业', 'N/A'),
        '总股本': info_dict.get('总股本', 'N/A'),
        '流通股本': info_dict.get('流通股本', 'N/A'),
        '总市值': info_dict.get('总市值', 'N/A'),
        '市盈率': info_dict.get('市盈率', 'N/A'),
        '市净率': info_dict.get('市净率', 'N/A')
    }
    
    for key, value in key_info.items():
        print(f"   {key}: {value}")
        
except Exception as e:
    print(f"❌ Error getting stock info: {e}")


In [ ]:
# Get historical data for a specific stock
symbol = '000001'  # 平安银行
start_date = '20240101'  # Start of 2024
end_date = datetime.now().strftime('%Y%m%d')  # Today

print(f"📈 Getting historical data for {symbol}...")
print(f"📅 Period: {start_date} to {end_date}")

try:
    # Get daily historical data
    hist_data = ak.stock_zh_a_hist(symbol=symbol, period='daily', start_date=start_date, end_date=end_date)
    
    print(f"✅ Successfully retrieved {len(hist_data)} trading days")
    print(f"📊 Data shape: {hist_data.shape}")
    print(f"📅 Columns: {list(hist_data.columns)}")
    
    # Display first few rows
    print("\n📈 First 5 trading days:")
    print(hist_data.head())
    
    # Display last few rows
    print("\n📈 Last 5 trading days:")
    print(hist_data.tail())
    
except Exception as e:
    print(f"❌ Error getting historical data: {e}")


In [ ]:
# Example: Build a stock screener
print("🔍 Building a Stock Screener...")

# Get real-time data
rt_data = ak.stock_zh_a_spot_em()

# Filter criteria
min_price = 10.0
max_price = 50.0
min_volume = 1000000  # 1M shares
min_gain = 2.0  # 2% gain

print(f"\n📊 Screening criteria:")
print(f"   Price range: {min_price} - {max_price} CNY")
print(f"   Minimum volume: {min_volume:,} shares")
print(f"   Minimum gain: {min_gain}%")

# Apply filters
screened_stocks = rt_data[
    (rt_data['最新价'] >= min_price) &
    (rt_data['最新价'] <= max_price) &
    (rt_data['成交量'] >= min_volume) &
    (rt_data['涨跌幅'] >= min_gain)
]

print(f"\n✅ Found {len(screened_stocks)} stocks matching criteria")

if len(screened_stocks) > 0:
    # Sort by gain percentage
    screened_stocks = screened_stocks.sort_values('涨跌幅', ascending=False)
    
    print("\n🎯 Top matches:")
    display_cols = ['代码', '名称', '最新价', '涨跌幅', '成交量', '成交额']
    print(screened_stocks[display_cols].head(10).to_string(index=False))
else:
    print("   No stocks found matching all criteria")


In [ ]:
# Example: Robust data fetching with error handling
import time
from typing import Optional, Dict, List

def safe_get_stock_info(symbol: str, max_retries: int = 3) -> Optional[Dict]:
    """Safely get stock information with retries"""
    for attempt in range(max_retries):
        try:
            print(f"   Attempt {attempt + 1}/{max_retries} for {symbol}")
            
            # Get stock info
            stock_info = ak.stock_individual_info_em(symbol=symbol)
            
            if stock_info.empty:
                print(f"   ⚠️  No data found for {symbol}")
                return None
            
            # Convert to dict format
            info_dict = {}
            for _, row in stock_info.iterrows():
                info_dict[row['item']] = row['value']
            
            return {
                'symbol': symbol,
                'name': info_dict.get('股票名称', ''),
                'sector': info_dict.get('所属行业', ''),
                'market_cap': info_dict.get('总市值', 0),
                'pe_ratio': info_dict.get('市盈率', 0),
                'pb_ratio': info_dict.get('市净率', 0)
            }
            
        except Exception as e:
            print(f"   ❌ Attempt {attempt + 1} failed: {e}")
            if attempt < max_retries - 1:
                time.sleep(1)  # Wait before retry
            else:
                print(f"   ❌ All attempts failed for {symbol}")
                return None

# Test the robust function
print("🔒 Testing robust data fetching:")
test_symbols = ['000001', '000002', '999999']  # Include a non-existent symbol

for symbol in test_symbols:
    print(f"\n📊 Processing {symbol}:")
    result = safe_get_stock_info(symbol)
    if result:
        print(f"   ✅ Success: {result['name']} - {result['sector']}")
    else:
        print(f"   ❌ Failed to get data for {symbol}")
